In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

## With the keywords loaded and exceptions

In [2]:
# import pandas as pd
# import re
# from tqdm import tqdm
#
# # --- CONFIG ---
# INPUT_FILE = "olx_land_sale.csv"
# OUTPUT_FILE = "dataset_with_price_text1.csv"
# TEXT_COLUMN = "description_"
# CONTEXT_WORDS = 3  # Number of words before and after
#
# def extract_price_with_context(text):
#     """
#     Extract price per land unit with 3 words before and after for context.
#     Returns None if no valid price pattern found.
#     """
#     if not isinstance(text, str) or text.strip() == "":
#         return None
#
#     # Clean up HTML tags but preserve text
#     text = re.sub(r'<br\s*/?>', ' ', text)
#     text = re.sub(r'<[^>]+>', '', text)
#     text = ' '.join(text.split())  # Normalize whitespace
#
#     # Patterns here:
#     patterns = [
#         #1: "22 000 за сотку" / "15000 за сотку"
#         r'(\d{3,}[\s,]*\d*)\s*(?:у\.е\.|usd|\$|доллар|dollar|ming|минг)?\s*(за|per|for)\s*(сотку|sotku|sotix|sot|сотих|сот|гектар|gektar)',
#
#         # 2: "сотиги 5500$" / "sotixi 20000"
#         r'(сотих[иы]|sotix[iy]|сот[иы]г[иы])\s*(\d{3,}[\s,]*\d*)\s*(?:\$|у\.е\.|usd|ming|минг|dollar)?',
#
#         # 3: "1 сотих 80.000 u.e" / "1 sotix 50000$"
#         r'(1\s*(?:сотку|сотих|сотка|sotix|sotku|sot))\s*(\d{3,}[\s,\.]*\d*)\s*(?:\$|у\.е\.|usd|ming|минг)?',
#
#         # 4: "цена 22 000 за сотку"
#         r'(цена|нарх[иы]?|price|cost)\s*:?\s*(\d{3,}[\s,]*\d*)\s*(?:\$|у\.е\.|usd|ming|минг)?\s*(за|per)\s*(сотку|sotix|сот)',
#
#         # 5: "5000 мингдан" (Uzbek "from price")
#         r'(\d{3,}[\s,]*\d*)\s*(ming|минг)(дан)',
#
#         # 6: "сотку: 15000" or "за сотку 20000"
#         r'(за|per)?\s*(сотку|сотих|sotix|sotku|сот)\s*[:-]?\s*(\d{4,}[\s,]*\d*)\s*(?:\$|у\.е\.|ming|минг|usd)?',
#     ]
#
#     best_match = None
#     best_match_pos = -1
#
#     for pattern in patterns:
#         matches = list(re.finditer(pattern, text, re.IGNORECASE))
#         for match in matches:
#             # Get the full matched text
#             matched_text = match.group(0)
#
#             # Filter out matches that are too short or just numbers
#             if len(matched_text.strip()) < 5:
#                 continue
#
#             # Check if this match contains actual price indicators
#             has_price_word = any(word in matched_text.lower() for word in
#                 ['сотку', 'sotix', 'сот', 'за', 'ming', 'минг', 'цена', 'нарх'])
#
#             if not has_price_word:
#                 continue
#
#             # Extract numbers and verify they're realistic prices
#             numbers = re.findall(r'\d+', matched_text.replace(' ', '').replace(',', ''))
#             if numbers:
#                 main_number = max(numbers, key=lambda x: len(x))
#                 # Filter unrealistic prices (too small)
#                 if len(main_number) < 3:
#                     continue
#
#             # Get context: 3 words before and after
#             start_pos = match.start()
#             end_pos = match.end()
#
#             # Find word boundaries before match
#             before_text = text[:start_pos]
#             before_words = before_text.split()[-CONTEXT_WORDS:]
#
#             # Find word boundaries after match
#             after_text = text[end_pos:]
#             after_words = after_text.split()[:CONTEXT_WORDS]
#
#             # Build context string
#             context = ' '.join(before_words + [matched_text] + after_words)
#
#             # Keep the first good match found (or prioritize by position)
#             if best_match is None or start_pos < best_match_pos:
#                 best_match = context.strip()
#                 best_match_pos = start_pos
#
#     return best_match if best_match else None
#
# # --- LOAD DATA ---
# print(" Loading data...")
# df = pd.read_csv(INPUT_FILE)
# total_rows = len(df)
# print(f"   Loaded {total_rows:,} rows")
#
# # --- PROCESS ---
# print(f"\n⚙ Processing all rows...")
# print(f"   Extracting price + {CONTEXT_WORDS} words before/after...")
# tqdm.pandas(desc="Extracting prices")
# df["price_text"] = df[TEXT_COLUMN].progress_apply(extract_price_with_context)
#
# # --- SAVE ---
# print(f"\n Saving results...")
# df.to_csv(OUTPUT_FILE, index=False)
#
# # --- STATISTICS ---
# found_count = df["price_text"].notna().sum()
# not_found = total_rows - found_count
#
# print(f"\n COMPLETE!")
# print(f"   ✓ Found prices: {found_count:,} rows ({found_count/total_rows*100:.1f}%)")
# print(f"   ✗ No price found: {not_found:,} rows ({not_found/total_rows*100:.1f}%)")
# print(f"    Saved to: {OUTPUT_FILE}")
#
# # --- SAMPLE RESULTS ---
# print("\n Sample extractions (first 15 with prices):")
# samples = df[df["price_text"].notna()].head(15)
# for idx, row in samples.iterrows():
#     print(f"\n   Row {idx}:")
#     print(f"   → {row['price_text']}")
#
# # --- SHOW ROWS WITHOUT PRICES ---
# print("\n\n Sample rows where NO price was found (first 5):")
# no_price = df[df["price_text"].isna()].head(5)
# for idx, row in no_price.iterrows():
#     preview = row[TEXT_COLUMN][:100].replace('\n', ' ')
#     print(f"\n   Row {idx}: {preview}...")
#
# print("\n\n Tips:")
# print("   • 'None' means no per-unit price was found")

## Converting land_area to simply "acre", and leaving it as float

In [3]:
# Regional average prices dictionary (in USD)
regional_prices = {
    "город Ташкент": 1251.69,  # Price segment: 10
    "Самаркандская область": 753.03,  # Price segment: 6
    "Ташкентская область": 695.25,  # Price segment: 5
    "Навоийская область": 583.55,  # Price segment: 4
    "Наманганская область": 470.91,  # Price segment: 2
    "Джизакская область": 464.21,  # Price segment: 2
    "Сурхандарьинская область": 455.77,  # Price segment: 2
    "Бухарская область": 454.60,  # Price segment: 2
    "Ферганская область": 444.84,  # Price segment: 2
    "Кашкадарьинская область": 443.78,  # Price segment: 2
    "Сырдарьинская область": 436.73,  # Price segment: 2
    "Хорезмская область": 397.03,  # Price segment: 2
    "Республика Каракалпакстан": 390.10  # Price segment: 1
}

# df['price_per_m2'] = (df['price'] / df['land_area']).round(2)

In [4]:
# df["land_area"] = (
#     df["land_area"]
#     .astype(str)                             # ensure string type
#     .str.replace("соток", "", regex=False)   # remove the word
#     .str.replace(" ", "", regex=False)        # remove inner spaces (like in '1 000')
#     .str.strip()                              # trim spaces
#     .replace("", None)                        # replace empty strings
#     .astype(float)                            # convert to float
# )
# df['land_area'].head()

In [5]:
# df["price_per_area"] = df["price"] / df["land_area"]
# df['price_per_area'].value_counts()


In [6]:
# df['purpose'].value_counts()

## Main cleaning

In [7]:
orig = pd.read_csv('olx_land_sale_orig.csv')
df = orig.copy()
df.rename(columns={'id_': 'id', 'url_':'url', 'title_': 'title', 'description_': 'description', 'status_': 'status', 'business_': 'business', 'last_refresh_time_': 'last_refresh_time', 'time__': 'time', 'price': 'price_usd', 'price_converted_value': 'price_uzs'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 39 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              22000 non-null  int64  
 1   url                             22000 non-null  object 
 2   title                           22000 non-null  object 
 3   description                     22000 non-null  object 
 4   business                        22000 non-null  int64  
 5   user_name                       22000 non-null  object 
 6   category_id                     22000 non-null  int64  
 7   category_type                   22000 non-null  object 
 8   location_region_name            22000 non-null  object 
 9   location_city_name              22000 non-null  object 
 10  location_district_name          21996 non-null  object 
 11  last_refresh_time               22000 non-null  object 
 12  created_time_                   

C:\Users\Saidakbar\AppData\Local\Temp\ipykernel_17448\1811184001.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  orig = pd.read_csv('olx_land_sale_orig.csv')


In [8]:
df.head()

,id,url,title,description,business,user_name,category_id,category_type,location_region_name,location_city_name,...,price_uzs,price_converted_currency,price_previous_value,price_converted_previous_value,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,https://www.olx.uz/d/obyavlenie/kuruk-er-sotil...,Курук ер сотилади 4 соту свет газ Сув интернет...,Ассалому алейкум Назарбекда<br />\n тукимачи м...,0,998992332145,10,real_estate,Ташкентская область,Назарбек,...,2.718807e+08,UZS,NaN,NaN,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,https://www.olx.uz/d/obyavlenie/prodaetsya-uch...,"Продаётся участок в центре, 2.5 сотки,вдоль до...","ПРОДАЖА УЧАСТКА <br />\nРайон: Яккасарайский,<...",1,Nigina,10,real_estate,Ташкентская область,Ташкент,...,3.304886e+09,UZS,NaN,NaN,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,https://www.olx.uz/d/obyavlenie/uch-kahramonda...,Уч кахрамондан 4 км кирганда Гулистонда 2 ёки ...,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,Акбар Исмаилов 2111,10,real_estate,Ташкентская область,Ташкент,...,2.764168e+08,UZS,NaN,NaN,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,https://www.olx.uz/d/obyavlenie/kibray-tsentr-...,Кибрай центр Продается земельный участок под с...,Продаётся земельный участок под строительство...,1,Хамза,10,real_estate,Ташкентская область,Кибрай,...,2.821104e+08,UZS,NaN,NaN,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,https://www.olx.uz/d/obyavlenie/srochno-uy-uri...,СРОЧНО!!! уй қуриш учун ер сотилади.,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,Алишер,10,real_estate,Ташкентская область,Кибрай,...,5.801046e+08,UZS,NaN,NaN,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


In [9]:
df['offer_type_'].value_counts()

offer_type_
offer    22000
Name: count, dtype: int64

In [10]:
cols_to_drop = ['url', 'title', 'user_name', 'category_id', 'last_refresh_time', 'created_time_',
                'valid_to_time_', 'pushup_time_', 'status', 'map_lat', 'map_lon', 'offer_type_', 'price_previous_value',
                'price_converted_previous_value']
df.drop(cols_to_drop, axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         22000 non-null  int64  
 1   description                22000 non-null  object 
 2   business                   22000 non-null  int64  
 3   category_type              22000 non-null  object 
 4   location_region_name       22000 non-null  object 
 5   location_city_name         22000 non-null  object 
 6   location_district_name     21996 non-null  object 
 7   promotion_highlighted      22000 non-null  int64  
 8   promotion_urgent           22000 non-null  int64  
 9   promotion_top_ad           22000 non-null  int64  
 10  promotion_premium_ad_page  22000 non-null  int64  
 11  time                       22000 non-null  object 
 12  price_usd                  22000 non-null  float64
 13  price_currency             22000 non-null  obj

In [12]:
df['time'] = pd.to_datetime(df['time'], format='%d_%m_%Y_%H_%M_%S').dt.normalize()

In [13]:
df.head()

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда<br />\n тукимачи м...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА <br />\nРайон: Яккасарайский,<...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


In [14]:
# Clean up of html leftovers
df['description'] = df['description'].str.replace(r'<br\s*/?>', '', regex=True)
df.head(5)

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_budget,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххала...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,0,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ра...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,0,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,0,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,0,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый


## Fill null values in the land_area, to catch its values from the description and fill it

In [15]:
# Clean extracted number: remove spaces, fix comma
def clean_number(val):
    if pd.isna(val):
        return val
    val = re.sub(r'\s+', '', str(val))  # remove all spaces
    val = val.replace(',', '.')         # standardize decimal
    return float(val)

# Use the improved pattern
num_pattern = r'\d{1,3}(?:\s\d{3})*(?:[.,]\d+)?'
pattern = rf'({num_pattern})\s*[сc][оo0][тtт]\w*'

mask = (
    df['land_area'].isnull() &
    df['description'].notnull() &
    df['description'].str.contains(pattern, regex=True, case=False)
)

extracted = df.loc[mask, 'description'].str.extract(pattern, flags=re.IGNORECASE)
df.loc[mask, 'land_area'] = extracted[0].apply(clean_number)
print(df[['land_area', 'description']].head(10))
print("Remaining nulls in land_area:", df['land_area'].isnull().sum())

     land_area                                        description
0      4 соток  Ассалому алейкум Назарбекда\n тукимачи маххала...
1   2.50 соток  ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n   Ра...
2     24 соток  Зудлик билан 2 ёки 4  сотих ер учаска сотилади...
3      7 соток  Продаётся  земельный участок под строительство...
4      6 соток  Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...
5     40 соток  Продаётся земельный участок под производство и...
6      8 соток  Продаётся земельный участок с кадастром \nКибр...
7    150 соток  Продается база с земельным участок \nПаркентск...
8  1 000 соток  Представляю  Вашему Вниманию  Инвестиционное п...
9    600 соток  - 6 соток 20/30 метр.\n- 2 кават, 6 хона, 4 са...
Remaining nulls in land_area: 7


C:\Users\Saidakbar\AppData\Local\Temp\ipykernel_17448\1181913159.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['description'].str.contains(pattern, regex=True, case=False)


## Test4 21k

In [16]:
SOTIX_REGEX = re.compile(
    r'''
    # Match number (supports: 6, 2.5, 4,5, 1 200, 10.000)
    (?P<number1>\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)
    \s*
    # Optional: range indicator (dash, "dan", "gacha", "–", etc.)
    (?:
        [-–—~]?\s*                     # dash or no dash
        (?:dan\s+)?                    # optional "dan"
        (?P<number2>\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)?
        \s*
    )?
    # Flexible "sot" unit (covers ALL variants in your data)
    [сc][оo0][тtт]                    # "sot" in Cyrillic/Latin/OCR
    \s*
    # Optional suffixes (ix, ok, ik, ux, их, ок, их, ихлик, etc.)
    [a-zа-я]*[ixkгоуhъь]*             # covers sotix, sotok, sotik, sutox, sotux, sotig‘, sutuk, so’tix, soʻtik, etc.
    ''',
    re.IGNORECASE | re.VERBOSE
)


def extract_sotix_from_desc(desc):
    if pd.isna(desc):
        return None

    match = SOTIX_REGEX.search(str(desc))
    if not match:
        return None

    def clean_num(s):
        return float(s.replace(' ', '').replace(',', '.'))

    try:
        low = clean_num(match.group('number1'))
        high_str = match.group('number2')
        if high_str:
            high = clean_num(high_str)
            return (low + high) / 2.0
        else:
            return low
    except (ValueError, TypeError):
        return None

In [17]:
# Extract sotix from description
df['extracted_land_area'] = df['description'].apply(extract_sotix_from_desc)

# Fill ONLY nulls in land_area
df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])

## Test3 2k

In [18]:
# # Main regex pattern
# SOTIX_PATTERN = r'''
# (\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)    # Group 1: number (6, 1 200, 2.5, 4,5)
# \s*                                    # optional space
# [-–—]?\s*                              # optional dash (for ranges)
# (?:dan\s+)?                            # optional "dan" (as in "6 dan 8")
# (\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)?    # Group 2: second number (for ranges)
# \s*                                    # optional space
# [сc][оo0][тtт]                        # flexible "sot" (cyrillic/latin/zero)
# \s*[a-zа-я]*[ixkгоуh]*                # optional endings: ix, ok, ik, ux, их, ок
# '''
#
# # Compile with verbose and ignore case
# compiled_pattern = re.compile(SOTIX_PATTERN, re.IGNORECASE | re.VERBOSE)

In [19]:
# def extract_sotix(desc):
#     if pd.isna(desc):
#         return None
#
#     # Search for pattern
#     match = compiled_pattern.search(str(desc))
#     if not match:
#         return None
#
#     num1_str = match.group(1)
#     num2_str = match.group(2)
#
#     # Clean number: remove spaces, fix comma
#     def clean_num(s):
#         return float(s.replace(' ', '').replace(',', '.'))
#
#     try:
#         low = clean_num(num1_str)
#         if num2_str:  # It's a range like "2-4" or "6 dan 8"
#             high = clean_num(num2_str)
#             return (low + high) / 2.0
#         else:
#             return low
#     except (ValueError, TypeError):
#         return None

In [20]:
# # Create extracted column
# df['extracted_land_area'] = df['description'].apply(extract_sotix)
#
# # Fill ONLY nulls in land_area
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])
#
# # Optional: drop helper column
# # df = df.drop(columns=['extracted_land_area'])

## Test2 1.5k

In [21]:
# # Pattern to match:
# # - plain numbers: "6 sotix"
# # - ranges: "2-4 sotix", "6 dan 8 gacha", "5-6sotok"
# range_pattern = r'(\d+(?:[.,]\d+)?)(?:\s*[-–—]\s*|\s+dan\s+)(\d+(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[a-zа-я]*[ixkгоуh]*'
#
# # Fallback: single number (your original case)
# single_pattern = r'(\d+(?:[.,]\d+)?(?:\s*\d{3})*(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[a-zа-я]*[ixkгоуh]*'
#
# def extract_sotix_robust(desc):
#     if pd.isna(desc):
#         return None
#
#     desc = str(desc)
#
#     # First: try to match a RANGE
#     range_match = re.search(range_pattern, desc, re.IGNORECASE)
#     if range_match:
#         try:
#             low = float(range_match.group(1).replace(',', '.'))
#             high = float(range_match.group(2).replace(',', '.'))
#             return (low + high) / 2.0
#         except ValueError:
#             pass
#
#     # Second: try single number (with space support: "1 000")
#     single_match = re.search(single_pattern, desc, re.IGNORECASE)
#     if single_match:
#         num_str = single_match.group(1).replace(' ', '').replace(',', '.')
#         try:
#             return float(num_str)
#         except ValueError:
#             pass
#
#     return None

In [22]:
# df['extracted_land_area'] = df['description'].apply(extract_sotix_robust)
#
# # Fill ONLY the NaNs in land_area
# df['land_area'] = df['land_area'].fillna(df['extracted_land_area'])

## Test1 500

In [23]:
# sotix_pattern = r'(\d{1,3}(?:\s?\d{3})*(?:[.,]\d+)?)\s*[сc][оo0][тtт]\s*[а-яa-z]*\s*[\d\w]*'
#
# # Explanation:
# # - \d{1,3}(?:\s?\d{3})* → matches 6, 1 200, 1200, 10 000
# # - [.,]\d+? → accepts 6.5 or 6,5
# # - [сc][оo0][тtт] → flexible "с/с", "о/o/0", "т/t"
# # - \s*[а-яa-z]*\s*[\d\w]* → matches endings like "их", "ok", "ka", "ux", "ig", etc.
#
#
# def extract_sotix(text):
#     if pd.isna(text):
#         return None
#     match = re.search(sotix_pattern, text, re.IGNORECASE)
#     if match:
#         num_str = match.group(1)
#         # Clean: remove spaces, replace comma with dot
#         num_clean = num_str.replace(' ', '').replace(',', '.')
#         try:
#             return float(num_clean)
#         except ValueError:
#             return None
#     return None

In [24]:
# # Create new column: extracted_land_area
# df['extracted_land_area'] = df['description'].apply(extract_sotix)
#
# # Optional: fill land_area if null (as you planned)
# mask = df['land_area'].isnull() & df['extracted_land_area'].notnull()
# df.loc[mask, 'land_area'] = df['extracted_land_area']
#
# # Check results
# print(df[['land_area', 'extracted_land_area', 'description']].head(10))
# print("Remaining nulls in land_area:", df['land_area'].isnull().sum())

In [25]:
df['description'][8]

'Представляю  Вашему Вниманию  Инвестиционное предложение \n\nпод  застройку  любого типа недвижимости, ориентир Турбаза  "Золотой корабль. Рядом речка "Чирчик.\n\nЗемельный участок с кадастром \nВдоль дороги трасса Ташкент-Чарвак, Бостанлыкский район, 1 линия!!\n\nИмеется проект для зоны отдыха. \n\nИдеальное расположение  для:\nГостиницы \nЗоны отдыха \nАпарт отеля \nКоттеджа\nДачу \nСанатория\nКлиника\n1.05 гектар\nнебольшой торг  имеется !\n\nЦена - 735000у.е.\n\nТел:+99893-801-99-57 Рустам \nСпециалист  по  недвижимости.'

In [26]:
df.head()

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_negotiable,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot,extracted_land_area
0,52891158,Ассалому алейкум Назарбекда\n тукимачи маххала...,0,real_estate,Ташкентская область,Назарбек,0,0,0,0,...,1,2.718807e+08,UZS,4 соток,Другое,В пригороде,"Электричество, Газ, Вода, Интернет",Да,NaN,4.0
1,51622740,"ПРОДАЖА УЧАСТКА \nРайон: Яккасарайский,\n Ра...",1,real_estate,Ташкентская область,Ташкент,Яккасарайский район,0,0,0,...,1,3.304886e+09,UZS,2.50 соток,Земля под строительство,"В городе, Вдоль трассы","Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый,2.5
2,48788994,Зудлик билан 2 ёки 4 сотих ер учаска сотилади...,0,real_estate,Ташкентская область,Ташкент,Юнусабадский район,1,0,1,...,1,2.764168e+08,UZS,24 соток,Земля под сад/огород,"В городе, На закрытой территории, В пригороде","Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый,4.0
3,56466143,Продаётся земельный участок под строительство...,1,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,1,2.821104e+08,UZS,7 соток,Земля под строительство,В городе,"Электричество, Вода, Газ, Канализация",Нет,Неделимый,7.0
4,55098422,Қибрай тумани Ёшлик маҳалласида жуда зўр жойда...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,1,5.801046e+08,UZS,6 соток,Земля под строительство,"Вдоль трассы, Возле водоема, реки, В дачном ма...","Электричество, Вода, Газ",Нет,Делимый,8.0


In [27]:
df['land_area'].value_counts()

land_area
6 соток        2913
4 соток        1875
8 соток        1455
10 соток       1160
5 соток        1032
               ... 
6.84 соток        1
5.25 соток        1
16.74 соток       1
4.33 соток        1
288 соток         1
Name: count, Length: 867, dtype: int64

In [28]:
df['extracted_land_area'].value_counts()

extracted_land_area
6.00     1399
4.00      905
8.00      622
10.00     525
5.00      428
         ... 
0.72        1
8.25        1
23.70       1
6.17        1
39.00       1
Name: count, Length: 461, dtype: int64

In [29]:
df.info()
df[['land_area', 'extracted_land_area']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         22000 non-null  int64         
 1   description                22000 non-null  object        
 2   business                   22000 non-null  int64         
 3   category_type              22000 non-null  object        
 4   location_region_name       22000 non-null  object        
 5   location_city_name         22000 non-null  object        
 6   location_district_name     21996 non-null  object        
 7   promotion_highlighted      22000 non-null  int64         
 8   promotion_urgent           22000 non-null  int64         
 9   promotion_top_ad           22000 non-null  int64         
 10  promotion_premium_ad_page  22000 non-null  int64         
 11  time                       22000 non-null  datetime64[ns]
 12  pric

,land_area,extracted_land_area
0,4 соток,4.0
1,2.50 соток,2.5
2,24 соток,4.0
3,7 соток,7.0
4,6 соток,8.0
...,...,...
21995,600 соток,NaN
21996,1 100 соток,NaN
21997,200 соток,NaN
21998,6 соток,NaN


In [30]:
df['land_area'].value_counts()
df['extracted_land_area'].value_counts()

extracted_land_area
6.00     1399
4.00      905
8.00      622
10.00     525
5.00      428
         ... 
0.72        1
8.25        1
23.70       1
6.17        1
39.00       1
Name: count, Length: 461, dtype: int64

In [31]:
df['extracted_land_area']

0        4.0
1        2.5
2        4.0
3        7.0
4        8.0
        ... 
21995    NaN
21996    NaN
21997    NaN
21998    NaN
21999    NaN
Name: extracted_land_area, Length: 22000, dtype: float64

In [32]:
df['land_area']=df['land_area'].str.split(' ').str[0]

In [33]:
df['extracted_land_area'].isna().sum()

np.int64(12785)

In [34]:
df = df[df['extracted_land_area'].isna()]

In [35]:
df['land_area'] = pd.to_numeric(df['land_area'],errors='coerce')


In [36]:
df['land_diff'] = df['extracted_land_area']-df['land_area']

In [40]:
pd.set_option('display.max_rows',100)
df.head(100)

,id,description,business,category_type,location_region_name,location_city_name,location_district_name,promotion_highlighted,promotion_urgent,promotion_top_ad,...,price_uzs,price_converted_currency,land_area,purpose,location,communications,comission,type_of_plot,extracted_land_area,land_diff
7,55776194,Продается база с земельным участок \nПаркентск...,1,real_estate,Ташкентская область,Паркент,0,0,0,0,...,8.290490e+09,UZS,150.00,Земля под строительство,"В городе, В пригороде",NaN,Нет,Неделимый,NaN,NaN
8,50900888,Представляю Вашему Вниманию Инвестиционное п...,1,real_estate,Ташкентская область,Искандар,0,0,0,0,...,9.342658e+09,UZS,1.00,Земля под строительство,"Вдоль трассы, Возле водоема, реки",NaN,Нет,Неделимый,NaN,NaN
10,56535796,Продается срочно земля 3 ст\nБуссув чаманзор м...,0,real_estate,Ташкентская область,Уртааул,0,0,0,0,...,2.054848e+08,UZS,3.00,Земля под строительство,"В пригороде, Вдоль трассы","Электричество, Телефон, Газ, Вода, Интернет",Нет,Неделимый,NaN,NaN
16,55400930,ПРОДАЖА ЗЕМЛИ\nПОД СТРОИТЕЛЬСТВО ЖК\nСЕРГЕЛИ Х...,1,real_estate,Ташкентская область,Ташкент,Сергелийский район,0,0,0,...,4.334062e+08,UZS,300.00,Земля под строительство,В городе,"Электричество, Канализация, Газ, Вода, Отопление",Нет,Неделимый,NaN,NaN
28,56036286,Дачний участка зангатинскийрайон катартал буш ...,0,real_estate,Ташкентская область,Ташкент,Шайхантахурский район,0,0,0,...,1.099724e+09,UZS,5.50,Земля под строительство,В пригороде,"Электричество, Интернет, Газ, Вода, Отопление",Нет,Неделимый,NaN,NaN
29,54458415,3sotik 5xona karopka ucaska srochna sotiladi \...,0,real_estate,Ташкентская область,Келес,0,0,0,0,...,6.320250e+08,UZS,3.00,Другое,В сельской местности,"Электричество, Газ, Вода",Нет,Делимый,NaN,NaN
30,52910865,Uy sotiladi Toshkent viloyati Yangiyol tumani ...,0,real_estate,Ташкентская область,Янгиюль,0,0,0,0,...,1.229062e+09,UZS,7.00,Другое,В пригороде,"Электричество, Телефон, Канализация, Газ, Вода...",Нет,Делимый,NaN,NaN
32,53049180,Bostonliq tumanida yer sotiladi tog' yonbag'ri...,0,real_estate,Ташкентская область,Газалкент,0,0,0,0,...,6.483450e+07,UZS,15.00,Земля под строительство,"В сельской местности, В предгорьях","Электричество, Газ, Вода",Нет,NaN,NaN,NaN
34,58265996,Продаётся земельный участок 1.5 га под произво...,1,real_estate,Ташкентская область,Зангиата,0,0,0,0,...,5.826015e+09,UZS,150.00,Земля под строительство,В пригороде,"Электричество, Телефон, Газ, Вода, Интернет",Нет,Неделимый,NaN,NaN
37,53389295,Yangi Uzbekistan park orqasi oz uyimizni yarim...,0,real_estate,Ташкентская область,Кибрай,0,0,0,0,...,5.041960e+08,UZS,8.00,Другое,В городе,"Электричество, Телефон, Газ, Вода, Интернет",Да,Делимый,NaN,NaN


In [43]:
df[['land_area','extracted_land_area','description']]

,land_area,extracted_land_area,description
7,150.0,NaN,Продается база с земельным участок \nПаркентск...
8,1.0,NaN,Представляю Вашему Вниманию Инвестиционное п...
10,3.0,NaN,Продается срочно земля 3 ст\nБуссув чаманзор м...
16,300.0,NaN,ПРОДАЖА ЗЕМЛИ\nПОД СТРОИТЕЛЬСТВО ЖК\nСЕРГЕЛИ Х...
28,5.5,NaN,Дачний участка зангатинскийрайон катартал буш ...
...,...,...,...
21995,600.0,NaN,Urganch tumani gʻallab mahallasi hayvat qishlo...
21996,1.0,NaN,Урганч шахар санотчилар кучасида 1100кв цемент...
21997,200.0,NaN,Гулистон махалласи Дурдона кучаси жасмин магаз...
21998,6.0,NaN,Срочно сотилоди бохосини\n галиштирип бараман ...


In [49]:
df.loc[21995]['description']

'Urganch tumani gʻallab mahallasi hayvat qishlogʻi Joy oʻrin sotiladi 600 kvadrat murojat uchun tel 880091005 narxiga keladon bolsak 30 million oxrgi baxosi'